In [13]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Import necessary libraries
%autoreload
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [15]:
energy_df = pd.read_csv('../data/energy_import_export.csv')
solar_df = pd.read_csv('../data/solar_self_consumption_main_building.csv')
met_df = pd.read_csv('../data/met_data.csv')

# Convert timestamp columns to datetime
energy_df['Tidspunkt'] = pd.to_datetime(energy_df['Tidspunkt'], format="mixed")
solar_df['starting_at'] = pd.to_datetime(solar_df['starting_at'], format='mixed')

# remove utc from the time string
met_df['starting_at'] = met_df['starting_at'].str.split('+').str[0]
met_df['starting_at'] = pd.to_datetime(met_df['starting_at'], format='mixed')

energy_df.rename(columns={
    'Energikilde': 'energy_source',
    'Retning': 'direction',
    'Målernavn': 'meter_name',
    'Måler-Id': 'meter_id',
    'Verdi': 'value',
    'Tidspunkt': 'timestamp'
}, inplace=True)


solar_df.rename(columns={
    'starting_at': 'timestamp'
}, inplace=True)
met_df.rename(columns={
    'starting_at': 'timestamp'
}, inplace=True)

meter_property_mapping = {
    '707057500042745649': 10724, #main building
    '707057500038344962': 10703, # building A
    '707057500085390523': 4462, # building B
    '707057500042201572': 4746 # building C
}
id_to_name_property_mapping= {
    10724: 'main building',
    10703: 'building A',
    4462: 'building B',
    4746: 'building C'
}
def get_name(id):
    return id_to_name_property_mapping[id]

energy_df['meter_id'] = energy_df['meter_id'].astype(str)
energy_df['meter_id'] = energy_df['meter_id'].str.strip()

energy_df['property_id'] = energy_df['meter_id'].map(meter_property_mapping)
energy_df['building'] = energy_df['property_id'].map(id_to_name_property_mapping)
merged_df = pd.merge(energy_df, met_df, on=['property_id', 'timestamp'], how='left')
# Separate import (consumption) and export (production)
import_df = merged_df[merged_df['direction'] == 'IMPORT']
export_df = merged_df[merged_df['direction'] == 'EXPORT']


In [16]:
main_building_import = import_df[import_df['property_id'] == 10724]
main_building_export = export_df[export_df['property_id'] == 10724]

main_building_df = pd.merge(main_building_import, solar_df, left_on='timestamp', right_on='timestamp', how='left')

# Replace NaN in solar_consumption with 0
main_building_df['solar_consumption'] = main_building_df['solar_consumption'].fillna(0)

# Calculate total consumption including self-consumed solar energy
main_building_df['total_consumption'] = main_building_df['value'] + main_building_df['solar_consumption']
# Get import data for reference buildings
ref_buildings = [10703, 4462, 4746]
ref_buildings_df = import_df[import_df['property_id'].isin(ref_buildings)]

# Aggregate data to daily or monthly consumption
main_daily_consumption = main_building_df.groupby(main_building_df['timestamp'].dt.date)['value'].sum()#.reset_index()
ref_daily_consumption = ref_buildings_df.groupby(['property_id', ref_buildings_df['timestamp'].dt.date])['value'].sum().reset_index()



# Add 'year' and 'week' columns
import datetime


main_building_df['year'] = main_building_df['timestamp'].dt.isocalendar().year
main_building_df['week'] = main_building_df['timestamp'].dt.isocalendar().week

ref_buildings_df['year'] = ref_buildings_df['timestamp'].dt.isocalendar().year
ref_buildings_df['week'] = ref_buildings_df['timestamp'].dt.isocalendar().week

# Weekly Aggregation for Main Building
main_weekly_consumption = main_building_df.groupby(['year', 'week'])['value'].sum()
main_weekly_consumption.index = pd.MultiIndex.from_tuples(main_weekly_consumption.index, names=['year', 'week'])

main_weekly_consumption = main_weekly_consumption.reset_index()
main_weekly_consumption['timestamp'] = main_weekly_consumption.apply(lambda x: datetime.date.fromisocalendar(int(x['year']), int(x['week']), 1), axis=1)
main_weekly_consumption['timestamp'] = pd.to_datetime(main_weekly_consumption['timestamp'])
main_weekly_consumption.index = main_weekly_consumption['timestamp']
main_weekly_consumption.drop(columns=['timestamp', "year", "week"], inplace=True)

# Weekly Aggregation for Reference Buildings
ref_weekly_consumption = ref_buildings_df.groupby(['property_id', 'year', 'week'])['value'].sum().reset_index()
# Optionally, create a representative timestamp for the week (e.g., the first day of the week)
# Convert 'year' and 'week' to integers
ref_weekly_consumption['year'] = ref_weekly_consumption['year'].astype(int)
ref_weekly_consumption['week'] = ref_weekly_consumption['week'].astype(int)

ref_weekly_consumption['timestamp'] = ref_weekly_consumption.apply(lambda x: datetime.date.fromisocalendar(int(x['year']), int(x['week']), 1), axis=1)
ref_weekly_consumption['timestamp'] = pd.to_datetime(ref_weekly_consumption['timestamp'])


/var/folders/b0/slnc__2x31jczblnsfgfmjw00000gn/T/ipykernel_54716/254535094.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ref_buildings_df['year'] = ref_buildings_df['timestamp'].dt.isocalendar().year
/var/folders/b0/slnc__2x31jczblnsfgfmjw00000gn/T/ipykernel_54716/254535094.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ref_buildings_df['week'] = ref_buildings_df['timestamp'].dt.isocalendar().week
